In [ ]:
%matplotlib inline

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

In [ ]:
def year_binner(year,val=10):
    return year - year%val

In [ ]:
keep_string=r"^.+_(PROPN|NOUN|ADJ|VERB||NUM|ADV)$"

In [ ]:
phrases=pd.read_pickle('/home/users0/pageljs/dh/repos/Compounding/datasets/phrases.pkl')

In [ ]:
context_df=phrases.groupby(['context','year'])['count'].sum().to_frame().reset_index()
context_df=context_df.loc[~context_df.context.str.contains('^@_')]
context_df=context_df.loc[context_df.context.str.match(keep_string)]
context_df.context=context_df.context.str.replace(r'.+_NUM','NUM',regex=True)
context_df["year"] = pd.to_numeric(context_df["year"], downcast="unsigned")
context_df=context_df.groupby(['context','year'])['count'].sum().to_frame().reset_index()
context_df

In [ ]:
context_df['decade']=year_binner(context_df['year'].values,10)
decade_df=context_df.groupby(['decade'])['count'].sum().to_frame().reset_index()
decade_df.set_index('decade',inplace=True)
decade_df['count_per']=decade_df['count']/decade_df['count'].sum()*100
decade_df

In [ ]:
plt.figure(figsize = (20,8))
ax = sns.barplot(x=decade_df.index, y="count_per", data=decade_df)
_=ax.set_xticklabels(ax.get_xticklabels(),rotation = 45)

In [ ]:
year_counts_df=context_df.groupby(['year'])['count'].sum().to_frame().reset_index()
year_counts_df['count_per']=year_counts_df['count']/year_counts_df['count'].sum()*100
year_counts_df

In [ ]:
plt.figure(figsize = (20,15))
ax = sns.barplot(x='year', y="count_per", data=year_counts_df)
_=ax.set_xticklabels(ax.get_xticklabels(),rotation = 45)
n = 5  # Keeps every 5th label
_=[l.set_visible(False) for (i,l) in enumerate(ax.xaxis.get_ticklabels()) if i % n != 0]

In [ ]:
presence_df=context_df.groupby(['context','decade']).size().to_frame().reset_index().context.value_counts().to_frame()
total_list=presence_df.loc[presence_df.context==20].index.to_list()
len(total_list)

In [ ]:
content_words_df=context_df.groupby(['context'])['count'].sum().to_frame().reset_index()
content_words_df

In [ ]:
context_top_50k=content_words_df.sort_values(by='count',ascending=False).head(50_000)['context'].to_list()
len(context_top_50k)

In [ ]:
len(set(context_top_50k).union(total_list))

In [ ]:
with open('/home/users0/pageljs/dh/repos/Compounding/context.pkl', 'wb') as f:
    pickle.dump(context_top_50k, f)

In [ ]:
plot_df=context_df.loc[context_df.context.isin(context_top_50k)].groupby(['year'])['count'].sum().to_frame().reset_index()
plot_df=plot_df.set_index('year')/year_counts_df.set_index('year')*100
plot_df

In [ ]:
plt.figure(figsize = (20,15))
ax = sns.barplot(x=plot_df.index, y="count", data=plot_df)
_=ax.set_xticklabels(ax.get_xticklabels(),rotation = 45)
n = 5  # Keeps every 5th label
_=[l.set_visible(False) for (i,l) in enumerate(ax.xaxis.get_ticklabels()) if i % n != 0]

In [ ]:
plot_df=context_df.loc[context_df.context.isin(context_top_50k)]['year'].value_counts().to_frame()
plot_df=plot_df/context_df['year'].value_counts().to_frame()*100
plot_df.columns=['types_per']
plot_df

In [ ]:
plt.figure(figsize = (20,15))
ax = sns.barplot(x=plot_df.index, y="types_per", data=plot_df)
_=ax.set_xticklabels(ax.get_xticklabels(),rotation = 45)
n = 5 
_=[lab.set_visible(False) for (i,lab) in enumerate(ax.xaxis.get_ticklabels()) if i % n != 0]

In [ ]:
context_df.loc[context_df.context.isin(context_top_50k)]['year'].value_counts().to_frame()